# Establish Communication with Vessel

In [1]:
import krpc
import time

from operator import attrgetter

conn = krpc.connect("MK-1")
vessel = conn.space_center.active_vessel
vessel_sit = conn.space_center.VesselSituation

# Initialize Auto-Pilot

In [2]:
vessel.auto_pilot.target_pitch_and_heading(90, 90)
vessel.auto_pilot.engage()
vessel.control.throttle = 1
time.sleep(1)

# Check for Science Objectives

In [3]:
# Start situation and biome streams
init_sit = vessel.situation

situation = conn.add_stream(getattr, vessel, 'situation')
biome = conn.add_stream(getattr, vessel, 'biome')

situation.start()
biome.start()

In [4]:
# Define methods for experiment management
transmissions = []

def get_experiments(vessel=vessel):
    sci_parts = vessel.parts.with_module('ModuleScienceExperiment')
    
    # Get only opearable experiments
    sci_parts = [part for part in sci_parts if not part.experiment.inoperable]
    
    # Get unique experiments
    experiments = []
    for part in sci_parts:
        unique = True
        for experiment in experiments:
            if part.title == experiment.title:
                unique = False
                break
                
        if unique:
            experiments.append(part) 
            
    return experiments

def run_experiment(sci_part):
    try:
        print(f"Running: {sci_part.title}")
        sci_part.experiment.run()

    except RuntimeError:
        print("Already had data. Attempting correction.")
        transmit(sci_part)
        time.sleep(2)
        run_experiment(sci_part)
        
def transmit(sci_part, transmissions=transmissions):
    # For data in sci_part
    for data in sci_part.experiment.data:
        # Transmit if data has value
        if data.transmit_value > 0:
            print(f"Transmitting: {sci_part.title}, {data.transmit_value}")
            transmissions += [(sci_part.title, data.transmit_value)]
            sci_part.experiment.transmit()

    
    # Reset experiment
    if not sci_part.experiment.inoperable:
        sci_part.experiment.reset()    

# Launch

In [ ]:
prev_sit_bio = None
transmitted = []
while situation() != vessel_sit.landed:
    # Get the current situation and biome in tuple
    sit_bio = (situation(), biome())
    
    # If the condition has not been visited yet,
    # get data
    if sit_bio != prev_sit_bio:
        print(f"Entering: {sit_bio}")
        experiments = get_experiments()
        
        for experiment in experiments:
            run_experiment(experiment)
            time.sleep(2)
            transmit(experiment)
            time.sleep(2)
        
        prev_sit_bio = sit_bio
   
    # If we have not launched now is the time
    if situation() == vessel_sit.pre_launch:
        vessel.control.activate_next_stage()

Entering: (<VesselSituation.pre_launch: 0>, 'Shores')
Running: Probodobodyne Stayputnik
Running: 2HOT Thermometer
Already had data. Attempting correction.
Running: 2HOT Thermometer
Already had data. Attempting correction.
Running: 2HOT Thermometer
Already had data. Attempting correction.
Running: 2HOT Thermometer
Already had data. Attempting correction.
Running: 2HOT Thermometer
Already had data. Attempting correction.
Running: 2HOT Thermometer
Already had data. Attempting correction.
Running: 2HOT Thermometer
Running: Mystery Goo™ Containment Unit 
Transmitting: Mystery Goo™ Containment Unit , 4.0320000648498535
Entering: (<VesselSituation.flying: 4>, 'Shores')
Running: Probodobodyne Stayputnik
Transmitting: Probodobodyne Stayputnik, 1.600000023841858
Running: 2HOT Thermometer
Transmitting: 2HOT Thermometer, 3.200000047683716
Running: Mystery Goo™ Containment Unit 
Transmitting: Mystery Goo™ Containment Unit , 4.0320000648498535
